<a href="https://colab.research.google.com/github/Angelawork/AI4Good_LiteraLingo_M3Project/blob/main/Datasets/Angela_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Idioms is a figure of speech
#Idioms: An idiom is an expression that cannot be understood from the meanings of its separate words but that has a separate meaning of its own.
#Many (although not all) idioms are examples of figurative language.


#Description: The idioms.csv file has 2 columns:ideom and definition
#Sources:
#https://huggingface.co/models?pipeline_tag=text2text-generation&sort=downloads&search=figurative-nlp
#1.figurative-nlp/t5-figurative-paraphras:https://huggingface.co/figurative-nlp/t5-figurative-paraphrase?text=No%2C+I%27d+say+this+was+dumbest
#This model can convert the figurative/metaphorical expression to the literal expression.
#2.figurative-nlp/sarcasm-hyperbole-humor-paraphrase:https://huggingface.co/figurative-nlp/sarcasm-hyperbole-humor-paraphrase 
#There is no description about this pretained model in Hugging face
#3.NLP(text2TextGeneration)

#batch size=128 or 256

In [ ]:
!pip install torch
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tensorflow as tf
from tensorflow import keras

!pip install --upgrade transformers
import transformers

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#There are 2 model:figurative-nlp/t5-figurative-paraphrase and figurative-nlp/sarcasm-hyperbole-humor-paraphrase
model_name1 = "figurative-nlp/t5-figurative-paraphrase"
model_name2 = "figurative-nlp/sarcasm-hyperbole-humor-paraphrase"

tokenizer = AutoTokenizer.from_pretrained(model_name1)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.7 MB/s eta 0:00:00


In [ ]:

model.eval()
idioms_df = pd.read_csv('/content/Untitled Folder/Proverbs.csv',encoding = "ISO-8859-1")
fig = list(idioms_df["Figurative sentence"][:100])
lit = list(idioms_df["Literal meaning"][:100])
input_ids = tokenizer("paraphrase the sentence : "+fig[0], padding=True, truncation=False,return_tensors="pt").input_ids
outputs = model.generate(input_ids,num_beams = 5)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("IN: ", fig[0])
print("OUT: ", result)
print("reference: ", lit[0])


In [ ]:
_, _, bert_score = bertscore.score([result], [lit[1]])
round(bert_score[0].tolist(),4)

# Evaluate

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sys
!pip install bert-score
import bert_score
import numpy as np
from transformers import logging

!pip install git+https://github.com/google-research/bleurt.git
import bleurt
from bleurt import score
logging.set_verbosity_error()

In [ ]:
bertscore = bert_score.BERTScorer(lang='en', rescale_with_baseline=True)
#checkpoint = '/content/bleurt_config.json'
#bleurt_scorer = score.BleurtScorer()
bleurt_scorer = load("bleurt", module_type="metric")

NameError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name1)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name1)

In [ ]:
def evaluate(link, amount):
  model.eval()
  idioms_df = pd.read_csv(link,encoding = "ISO-8859-1")
  fig = list(idioms_df["Figurative sentence"][:amount])
  lit = list(idioms_df["Literal meaning"][:amount])
  hyps=[]
  refs=lit
  counter=0
  for sentence in fig:
    counter+=1

    #insert model----------------------
    #model 1
    #input_ids = tokenizer("paraphrase the sentence : "+sentence, padding=True, truncation=False,return_tensors="pt").input_ids
    input_ids = tokenizer(sentence, padding=True, truncation=False,return_tensors="pt").input_ids
    outputs = model.generate(input_ids,num_beams = 5)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    #insert model----------------------

    if (counter%5 == 0):
      print("Generating at "+str(counter)+" times")
    hyps.append(result)
  return hyps, refs



def score(hyps, refs, type):
  if type=='bert':
    scores = []
    for l1,l2 in zip(hyps, refs):
      _, _, bert_score = bertscore.score([l1], [l2])
      scores.append(round(bert_score[0].tolist(),4))

  elif type=='bleu':
    #scores = bleurt_scorer.score(references=refs, candidates=hyps)
    scores = bleurt.compute(predictions=hyps, references=refs)
  print('The average score is {}'.format(np.mean(scores)))

In [ ]:
hyps, refs = evaluate("/content/Irony_new.csv",100)
score(hyps, refs,'bert')

Generating at 5 times
Generating at 10 times
Generating at 15 times
Generating at 20 times
Generating at 25 times
Generating at 30 times
Generating at 35 times
Generating at 40 times
Generating at 45 times
Generating at 50 times
Generating at 55 times
Generating at 60 times
Generating at 65 times
Generating at 70 times
Generating at 75 times
Generating at 80 times
Generating at 85 times
Generating at 90 times
Generating at 95 times
Generating at 100 times
The average score is 0.5778699999999999


## BLEURT scores returned by bleurt.score_files are usually in the range of [-inf, inf]. Higher scores indicate better similarity between the reference and candidate texts.

### The score ranges from 0 to 1, with a higher score indicating a higher similarity.

In [ ]:
hyps[:5]

["On my recommendation you aren't backing my project You're a fraud!",
 "I'd love to see it with the trophy.",
 "When she's ready to taste her food, you give her a taste of her own",
 "And that's a great thing.",
 "And it's worth fighting for."]

In [ ]:
hyps, refs = evaluate("/content/Metaphor.csv",100)
score(hyps, refs,'bleu')

Generating at 5 times
Generating at 10 times
Generating at 15 times
Generating at 20 times
Generating at 25 times
Generating at 30 times
Generating at 35 times
Generating at 40 times
Generating at 45 times
Generating at 50 times
Generating at 55 times
Generating at 60 times
Generating at 65 times
Generating at 70 times
Generating at 75 times
Generating at 80 times
Generating at 85 times
Generating at 90 times
Generating at 95 times
Generating at 100 times
The average score is 0.2700143152475357


# SPLIT dataset

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd

df=pd.read_csv('/content/Idioms.csv',encoding = "ISO-8859-1")

# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Split the test set further into validation and test sets
val_df, test_df = train_test_split(test_df, test_size=0.6, random_state=42)

# Print the sizes of the resulting datasets
print("total set size:", len(df))
print("Train set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Test set size:", len(test_df))

total set size: 144939
Train set size: 115951
Validation set size: 11595
Test set size: 17393
